In [ ]:
import json
import requests
import pandas as pd
from pandas.io.json import json_normalize

from bs4 import BeautifulSoup
import re

In [ ]:
def urls(url):
  r = requests.get(url)
  r.content
  soup= BeautifulSoup(r.content, 'html.parser')
  return soup

In [ ]:
def cinema_arr(n):
  debut_url = 'http://www.allocine.fr/salle/cinemas-pres-de-1157'
  if  n in range(21):
    code = n + 55
    fin_url = str(code)
    arr_url = debut_url+fin_url
    return arr_url

def salles_arr(arr_url):
  soup = urls(arr_url)
  
  salles = [i for i in soup.select("div>h2>a")]
  salles = pd.DataFrame(salles)
  display(salles)
  
def theater_choice(x,y):
  soup = urls(y)
  list_fin_url = [i.text.strip() for i in soup.select("div>h2>a")]
  fin_url = list_fin_url[x]
  regex = r"\d{4}"

  intermediate_url = [i['href'] for i in soup.select("div>h2>a")]

  list_fin_url = [re.search(regex,i).group(0) for i in intermediate_url]
  list_fin_url
  fin_url = list_fin_url[x]
  theater_unique_url = f'http://www.allocine.fr/seance/salle_gen_csalle=C{fin_url}.html'
  
  #print (theater_unique_url)
  return theater_unique_url
 
def theater_movie_list(x):
  soup = urls(x)
  movies_halles = [i.text.strip().lower() for i in soup.select("div>span>a")]
  display (pd.DataFrame(movies_halles,columns = ['select your movie : ']))
  halles_list = [i for i in soup.select("div>span>a")]
  movie_picked = input('pick a movie : ')
  b = "div#movie"
  d= ' span.hours-item-value'
  for i in range(len(movies_halles)):
    while movie_picked not in movies_halles: 
      print('I could not find the movie you picked. Make a new selection from the list.')
      movie_picked = input('pick a movie : ')
    if movie_picked == movies_halles[i]:
      id_s = [re.search('cfilm=(.*).html',i['href'])[1].split(',') for i in halles_list]
      c =(id_s[i][0])
      e=b+c+d
      times = [i.text for i in soup.select(e) ]
      time_list = times[:5]
      print('the next 5 show times today/tomorrow for',movie_picked,'are at :')
      display (pd.DataFrame(time_list, columns = [' ']))
      
def paris_cine():
  url = 'http://www.allocine.fr/salle/'
  
  soup = urls(url)
  
  villes = [i.text.strip('Paris ') for i in soup.select("div>p>a>span") if i.text.strip().endswith('ent')]
  df_arr = pd.DataFrame(villes, columns = ['select your area : '])
  
  display(df_arr)
  message = 'type a number between 1 and 20 included. \nNote there are no movie theaters in the 7ème arrondissement : '
  error_mess = 'only numbers between 1 and 20 included are accepted.'
  choice = input(message)
  while choice.isdigit() == False:
    print(error_mess)
    choice = input(message)
  choice = int(choice)
  while (choice >20) | (choice <1)| (choice == 7) :
    print(error_mess)
    choice = int(input(message))
    
  arr_url = cinema_arr(choice)
  soup = urls(arr_url)
  
  salles = [i.text.strip() for i in soup.select("div>h2>a")]
  df_salles = pd.DataFrame(salles, columns = ['Select your movie theater : '])
  
  display(df_salles)
  
  message2 = 'type a number between 1 and 7 included : '
  choice2 = input(message2)
  error_message = 'only numbers between 1 and 7 included are accepted : '
  while choice2.isdigit() == False:
    
    print(error_message)
    choice2 = input(message2)
  choice2 = int(choice2)
  while (choice2 >7) | (choice2 <1) :
    print(error_message)
    choice2 = int(input(message2))
    
  theater_url = theater_choice(choice2, arr_url)
  movie_list = theater_movie_list(theater_url)
  
paris_cine()